## Collect all prerequisites for theory steps

In [2]:
import pandas as pd

In [12]:
import requests

In [3]:
topic_statistics = '/home/anna/Documents/Diploma/topics_with_statistics/bigger_statistics_one_year.csv'

In [4]:
one_year_statistics_df = pd.read_csv(topic_statistics)

In [5]:
one_year_statistics_df.head()

,step_id,is_theory,text,seconds_to_complete,last_year_completion_rate,last_year_avg_like,last_year_completed_step_users_count,last_year_likes_count,last_year_topic_completion_rate,last_year_completed_topic_users_count,back_to_theory_times_per_user_session_avg_last_year,back_to_theory_users_%_last_year,cleaned_texts
0,4660,0,"""<p>Select correct code snippets with loops.</...",71.11,0.96,1.51,1394.0,180.0,NaN,NaN,NaN,NaN,Select correct code snippets with loops.\n \na...
1,13289,0,"""<p><p>Complete the <code class=\""language-jav...",32.23,0.95,1.87,629.0,47.0,NaN,NaN,NaN,NaN,Complete the @GetMapping annotation so that it...
2,13147,0,"""<p><a target=\""_blank\"" href=\""https://www.dr...",37.22,0.93,1.41,116.0,17.0,NaN,NaN,NaN,NaN,"Download a dataset and answer the question, h..."
3,5090,0,"""<p>If you rely on some dependency only during...",28.19,0.99,1.62,1569.0,87.0,NaN,NaN,NaN,NaN,If you rely on some dependency only during the...
4,12941,0,"""<p>What is the term for accessing a database ...",66.20,0.89,1.24,332.0,38.0,NaN,NaN,NaN,NaN,What is the term for accessing a database and ...


In [6]:
topic_statistics = one_year_statistics_df.drop(one_year_statistics_df[one_year_statistics_df.is_theory == 0].index)

In [7]:
topic_statistics.step_id.describe()

count     1321.000000
mean     10982.750189
std       4695.010154
min       1933.000000
25%       7112.000000
50%      10832.000000
75%      15269.000000
max      19087.000000
Name: step_id, dtype: float64

In [8]:
def get_topic_id(step_id, link='https://hyperskill.org/api'):
    response = requests.get(f'{link}/steps/{step_id}')
    return response.json()['steps'][0]['topic']

In [9]:
def get_immediate_prereq(topic_id, link='https://hyperskill.org/api'):
    response = requests.get(f'{link}/topics/{topic_id}')
    try:
        result = response.json()['topics'][0]['prerequisites']
    except:
        print(response.json())
        raise
    return result

In [16]:
def get_all_prereq(topic_id, dict_with_topics_info, link='https://hyperskill.org/api', _print=False):
    if topic_id in dict_with_topics_info:
        if _print: print(f'topic {topic_id} is already in dict')
        return dict_with_topics_info[topic_id]
    try:
        list_prereq = get_immediate_prereq(topic_id)
        if _print: print(f'immediate prereq of {topic_id} are {list_prereq}')
    except:
        print(topic_id)
        if topic_id in {1032, 350, 1075, 1174, 1114}:  # these topics are not found
            if _print: print(f'passing because of {topic_id}')
            list_prereq = []
            pass
        else:
            raise
    if not list_prereq:  # list is empty
        dict_with_topics_info[topic_id] = []
        if _print: print(f'prereq for {topic_id} are empty')
        return []
    
    set_prereq = set(list_prereq)
        
    for el in list_prereq:
        if _print: print(f'getting prereq of {el}')
            
        new_prereq = get_all_prereq(el, dict_with_topics_info, _print=_print)  # recursion
        dict_with_topics_info[el] = new_prereq
        
        set_prereq = set_prereq.union(new_prereq)
    return list(set_prereq)      

In [13]:
step_id_topic_id_mapping = {}
topic_id_step_id_mapping = {}

for i, s_id in enumerate(topic_statistics.step_id.values):
    if i % 10 == 0:
        print(i)
        
    corresponding_topic_id = get_topic_id(s_id)
    
    step_id_topic_id_mapping[s_id] = corresponding_topic_id
    topic_id_step_id_mapping[corresponding_topic_id] = s_id

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320


In [17]:
import pickle

In [18]:
pickle.dump(step_id_topic_id_mapping, open("step_id_topic_id_mapping.pkl", "wb"))
pickle.dump(topic_id_step_id_mapping, open("topic_id_step_id_mapping.pkl", "wb"))

### Collect all prerquisites

In [22]:
#dict_w_topic_prerequisites = dict()

for i, topic_id in enumerate(topic_id_step_id_mapping):
    if topic_id in dict_w_topic_prerequisites:
        continue
    print(i)
        
    prerequisites = get_all_prereq(topic_id, dict_w_topic_prerequisites)
    dict_w_topic_prerequisites[topic_id] = prerequisites

1169
1170
1171
1173
1175
1176
1178
1180
1182
1183
1184
1190
1191
1192
1193
1202
1204
1205
1209
1213
1215
1217
1224
1227
1228
1229
1232
1235
1239
1240
1242
1244
1247
1248
1250
1252
{'detail': 'Not found.'}
1114
1256
1258
1261
1264
1267
1269
1271
1272
1273
1274
1278
1279
1280
1282
1283
1284
1291
1296
1297
1300
1301
1312
1313


In [ ]:
# 1032, 350, 1075, 1174, 1114 -- errors

In [24]:
get_all_prereq(1032, dict(), _print=True)

{'detail': 'Not found.'}
1032
passing because of 1032
prereq for 1032 are empty


[]

In [25]:
pickle.dump(dict_w_topic_prerequisites, open("topic_id_all_prereq_mapping.pkl", "wb" ))

### Collect direct prerequisites

In [26]:
direct_prereqs = dict()

for i, s_id in enumerate(topic_statistics.step_id.values):
        
    if i % 10 == 0:
        print(i)
        
    corresponding_topic_id = step_id_topic_id_mapping[s_id]
    
    if corresponding_topic_id in direct_prereqs:
        continue
    try:
        direct_prerequisites = get_immediate_prereq(corresponding_topic_id)
    except:
        print(s_id, corresponding_topic_id)
    
    direct_prereqs[corresponding_topic_id] = direct_prerequisites
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
{'detail': 'Not found.'}
10924 1032
180
190
200
210
220
230
240
250
260
270
14804 1542
280
290
300
310
320
330
340
350
360
370
380
390
400
410
{'detail': 'Not found.'}
5507 350
420
430
11461 1081
440
450
460
470
480
490
500
510
520
530
{'detail': 'Not found.'}
11408 1075
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
14235 1468
770
780
790
800
810
820
830
840
{'detail': 'Not found.'}
12155 1174
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
15420 1625
1190
1200
1210
1220
1230
1240
1250
{'detail': 'Not found.'}
11742 1114
1260
1270
1280
1290
1300
1310
1320


In [27]:
# for these there was an error with 'not found'
print(direct_prereqs[1032])  # most probably there's the prerequisites of a previous topic
print(direct_prereqs[350])   # need to delete them from the dictionary
print(direct_prereqs[1075])
print(direct_prereqs[1174])  
print(direct_prereqs[1114])

# for these the problem was probably temporary
print(direct_prereqs[1542])
print(direct_prereqs[1081])
print(direct_prereqs[1468])
print(direct_prereqs[1625])

[663, 1745, 703, 706]
[1083]
[17, 24]
[362]
[876, 934, 1389]
[211, 441]
[152, 307]
[164, 167, 48, 51]
[149, 876]


In [28]:
for topic_id in [1032, 350, 1075, 1174, 1114]:  # for these there was an error with 'not found'
    print(topic_id)
    try:
        direct_prerequisites = get_immediate_prereq(topic_id)
        print(direct_prerequisites)
    except:
        print('error')
        
# these are deleted topics that were mentioned as prerequisites, they are now disabled

1032
{'detail': 'Not found.'}
error
350
{'detail': 'Not found.'}
error
1075
{'detail': 'Not found.'}
error
1174
{'detail': 'Not found.'}
error
1114
{'detail': 'Not found.'}
error


In [29]:
for topic_id in [1542, 1081, 1468, 1625]:  # for these the problem was probably temporary
    print(topic_id)
    try:
        direct_prerequisites = get_immediate_prereq(topic_id)
        print(direct_prerequisites)
    except:
        print('error')

1542
[1541]
1081
[1029]
1468
[1467, 2029]
1625
[432, 463]


In [30]:
# now we delete from the dictionary those that had 'not found' errors
# and update values for the two with the other kind of error
for topic in [1032, 350, 1075, 1174, 1114]:
    del direct_prereqs[topic]
    
for topic in [1542, 1081, 1468, 1625]:
    direct_prerequisites = get_immediate_prereq(topic)
    direct_prereqs[topic] = direct_prerequisites

In [31]:
pickle.dump(direct_prereqs, open("topic_id_direct_prereq_mapping.pkl", "wb" ))